# Gerador de HTML com EmulatorJS + ROM

## Setup

In [33]:
import base64
import requests
import io
from pathlib import Path
from pydash import py_


html_header = """<!doctype html>
<html lang="pt-br">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>{title}</title>
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-aFq/bzH65dt+w6FI2ooMVUpc+21e0SRygnTpmBvdBgSdnuTN7QbdgL+OapgHtvPp" crossorigin="anonymous">
<style>
* {{
  box-sizing: content-box;
}}
.game-container {{
    width: 640px;
    height: 480px;
    max-width: 100%;
    margin: auto;
}}
</style>
</head>
<body>
<br>"""

html_footer = """
<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha2/dist/js/bootstrap.bundle.min.js" integrity="sha384-qKXV1j0HvMUeCBQ+QVp7JcfGl760yU08IQ+GpUo5hlbpg51QRiuqHAJz8+BrxE/N" crossorigin="anonymous"></script>
</body>
</html>"""

html_base_1 = """
<div class="container">
    <h2 class="text-center">{title}</h2>
    <div class="game-container">
        <div id="game"></div>
    </div>
    <script type="text/javascript">
        var EJS_player = "#game";
        var EJS_core = "{core_name}";
        var EJS_lightgun = false;
        var EJS_biosUrl = "";
        var EJS_gameUrl = "{game_url}";
        var EJS_pathtodata = "https://rawcdn.githack.com/EmulatorJS/EmulatorJS/main/data/";
    </script>
    <script src="https://rawcdn.githack.com/EmulatorJS/EmulatorJS/main/data/loader.js"></script>
</div>"""


html_base_2 = """
<div class="container">
    <h2 class="text-center">{title}</h2>
    <div class="game-container">
        <div id="game"></div>
    </div>
    <script type="text/javascript">
        var byteCharacters = atob("{bin_b64}");
        var byteNumbers = new Array(byteCharacters.length);
        for (let i = 0; i < byteCharacters.length; i++) {{
            byteNumbers[i] = byteCharacters.charCodeAt(i);
        }}
        var EJS_player = "#game";
        var EJS_core = "{core_name}";
        var EJS_lightgun = false;
        var EJS_biosUrl = "";
        var EJS_gameUrl = URL.createObjectURL(new Blob([Uint8Array.from(byteNumbers)]));
        var EJS_pathtodata = "https://rawcdn.githack.com/EmulatorJS/EmulatorJS/main/data/";
    </script>
    <script src="https://rawcdn.githack.com/EmulatorJS/EmulatorJS/main/data/loader.js"></script>
</div>"""


class WebGame:
    def __init__(self):
        self.html = {
            "header": html_header,
            "footer": html_footer,
            "base_1": html_base_1,
            "base_2": html_base_2,
        }
        self.cores_allowed = [
            "nes",
            "snes",
            "gb",
            "gbc",
            "gba",
            "segaMD",
            "fbalpha2012_cps1",
        ]

    def _download_rom(self, url: str) -> io.BytesIO:
        file = io.BytesIO()
        with requests.get(url, stream=True) as res:
            res.raise_for_status()
            for chunk in res.iter_content(chunk_size=8192): 
                file.write(chunk)
        file.seek(0)
        return file

    def _generate_content_1(self, core_name: str, game_url: str, title: str) -> str:
        return self.html["base_1"].format(game_url=game_url, title=title, core_name=core_name)

    def _generate_content_2(self, core_name: str, game_url: str, title: str) -> str:
        file = self._download_rom(url=game_url)
        bin_b64 = base64.b64encode(file.read()).decode()
        return self.html["base_2"].format(game_url=game_url, title=title, core_name=core_name, bin_b64=bin_b64)

    def _generate_content_3(self, core_name: str, game_url: str, title: str) -> str:
        if not Path(game_url).exists():
            raise Exception(f"Arquivo '{game_url}' não existe!")
        with open(file=game_url, mode="rb") as file:
            bin_b64 = base64.b64encode(file.read()).decode()
        return self.html["base_2"].format(game_url=game_url, title=title, core_name=core_name, bin_b64=bin_b64)

    def make_html(self, core_name: str, game_url: str, title: str, b64_rom: bool = True, local_file: bool = False) -> str:
        if core_name not in self.cores_allowed:
            raise Exception(f"Core '{core_name!s}' não permitido! São permitidos: {self.cores_allowed!s}")
        if local_file is True:
            content = self._generate_content_3(game_url=game_url, core_name=core_name, title=title)
        elif b64_rom is True:
            content = self._generate_content_2(game_url=game_url, core_name=core_name, title=title)
        else:
            content = self._generate_content_1(game_url=game_url, core_name=core_name, title=title)
        header = self.html["header"].format(title=title)
        footer = self.html["footer"]
        html_content = f"{header}{content}{footer}"
        file = Path.cwd().joinpath("files/webgames").joinpath(py_.snake_case(title) + ".html")
        with open(file=str(file.resolve()), mode="w") as fileout:
            fileout.write(html_content)
        return f"Foi gerado o arquivo: '{file.resolve()!s}'."

webgame = WebGame()
webgame

## Gerar HTML

### NES - Metroid

In [14]:
webgame.make_html(
    game_url="https://myrient.erista.me/files/No-Intro/Nintendo%20-%20Nintendo%20Entertainment%20System%20(Headered)/Metroid%20(USA).zip",
    core_name="nes",
    title="NES - Metroid"
)

Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/nes_metroid.html'.


### GBA - Metroid Zero Mission

In [19]:
webgame.make_html(
    game_url="https://myrient.erista.me/files/No-Intro/Nintendo%20-%20Game%20Boy%20Advance/Metroid%20-%20Zero%20Mission%20(USA).zip",
    core_name="gba",
    title="GBA - Metroid Zero Mission"
)

Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/gba_metroid_zero_mission.html'.


### GBA - Metroid Fusion

In [20]:
webgame.make_html(
    game_url="https://myrient.erista.me/files/No-Intro/Nintendo%20-%20Game%20Boy%20Advance/Metroid%20Fusion%20(USA).zip",
    core_name="gba",
    title="GBA - Metroid Fusion"
)

Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/gba_metroid_fusion.html'.


### SNES - Super Metroid

In [18]:
webgame.make_html(
    game_url="https://myrient.erista.me/files/No-Intro/Nintendo%20-%20Super%20Nintendo%20Entertainment%20System/Super%20Metroid%20(Japan,%20USA)%20(En,Ja).zip",
    core_name="snes",
    title="SNES - Super Metroid"
)

Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/snes_super_metroid.html'.


### GB - Metroid 2

In [22]:
webgame.make_html(
    game_url="https://myrient.erista.me/files/No-Intro/Nintendo%20-%20Game%20Boy/Metroid%20II%20-%20Return%20of%20Samus%20(World).zip",
    core_name="gb",
    title="GB - Metroid 2"
)

Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/gb_metroid_2.html'.


### GBC - Metroid 2 DX (Color Hack)

In [34]:
webgame.make_html(
    game_url="/mnt/d/Games/RetroArch/roms/gameboy_color/Metroid II - Return of Samus (EJRTQ v1.3).zip",
    core_name="gbc",
    title="GBC - Metroid 2 DX",
    local_file=True
)

"Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/gbc_metroid_2_dx.html'."

### CPS1 - Cadillacs And Dinosaur

In [27]:
webgame.make_html(
    game_url=str(Path.cwd().joinpath("files/webgames").joinpath("dino.zip").resolve()),
    core_name="fbalpha2012_cps1",
    title="CPS1 - Cadillacs and Dinosaurs",
    local_file=True,
)

Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/cps_1_cadillacs_and_dinosaurs.html'.


### Mega Drive - Streets of Rage 2

In [30]:
webgame.make_html(
    game_url="https://myrient.erista.me/files/No-Intro/Sega%20-%20Mega%20Drive%20-%20Genesis/Streets%20of%20Rage%202%20(USA).zip",
    core_name="segaMD",
    title="Mega Drive - Streets of Rage 2",
)

"Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/mega_drive_streets_of_rage_2.html'."

### SNES - Top Gear

In [31]:
webgame.make_html(
    game_url="https://myrient.erista.me/files/No-Intro/Nintendo%20-%20Super%20Nintendo%20Entertainment%20System/Top%20Gear%20(USA).zip",
    core_name="snes",
    title="SNES - Top Gear",
)

"Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/snes_top_gear.html'."

### SNES - Top Gear 2

In [32]:
webgame.make_html(
    game_url="https://myrient.erista.me/files/No-Intro/Nintendo%20-%20Super%20Nintendo%20Entertainment%20System/Top%20Gear%202%20(USA).zip",
    core_name="snes",
    title="SNES - Top Gear 2",
)

"Foi gerado o arquivo: '/home/douglaspands/workspace/vscode-notebook/files/webgames/snes_top_gear_2.html'."